In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_DEV=1

import numpy as np

env: ANYWIDGET_DEV=1


In [2]:
from sequilt import Sequilt
from sequilt.model import LabelModel

In [3]:
from datasets import load_dataset
from sequilt.data import get_ids, get_tokenizer

ds = load_dataset("neuralbioinfo/bacterial_promoters")
tokenizer = get_tokenizer(type="dna", k=1)
ids, tokens = get_ids(
  ds["test_multispecies"]["segment"], tokenizer, max_tokens=32
)
labels = [
  LabelModel(value=value, name=name)
  for value, name in tokenizer._id_to_token.items()
  if value != 0
]

100%|██████████| 22582/22582 [00:00<00:00, 34781.35it/s]


In [4]:
from sequilt.model import Event, EventGraph

G = EventGraph(ids)

In [5]:
max_weight_edge = max(G.edges(data=True), key=lambda x: x[2]["weight"])

In [6]:
max_weight_edge

(Event(Position=0, Value=1, # Occurences=5705),
 Event(Position=31, Value=2, # Occurences=5531),
 {'weight': (31, 0.2771650696076659)})

In [11]:
from sequilt.dummy import dummy_labels, dummy_sequlets

w = Sequilt(dummy_sequlets, dummy_labels, width=800, height=200)
w

Sequilt(height=200, labels=[{'value': 0, 'name': 'A'}, {'value': 1, 'name': 'B'}, {'value': 2, 'name': 'C'}, {…